In [7]:
# !pip install --upgrade tensorflow --quiet
# !pip install keras_tuner --quiet
# !pip install tensorflow-io --quiet
# # Google colab modules
# from google.colab import drive
import sys, importlib

# # Mount drive
# drive.mount('/content/gdrive', force_remount=True)
ROOT_PATH = './'
# sys.path.append(ROOT_PATH)

import coremlv2 as core
core._init_ml()
# core._init_models()
# core.os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Reload coreml
importlib.reload(core)
import keras_tuner as kt

In [8]:
# Limiting GPU memory growth
gpus = core.tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            core.tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = core.tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs, ", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs,  1 Logical GPUs


### model_307_kt - world stock data (small)
best model trained with whole idx data.

In [2]:
kt_iter = '24'
ticker_group = ['wsd']
epochs = 20
max_epochs = 20
batch_size = 64

In [3]:
dataset_size = 'small'
shuffle_buffer_size = 1024
generator = False

train_inputs, train_labels = core.load_dataset_wsd_traintest(subset='training', dataset_size=dataset_size, ROOT_PATH='''H:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator)

validation_inputs, validation_labels = core.load_dataset_wsd_traintest(subset='validation', dataset_size=dataset_size, ROOT_PATH='''H:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator)

Total constituents: 22
Total constituents: 8


In [ ]:
tuner = kt.Hyperband(hypermodel=core.model_307_kt, objective='val_loss', max_epochs=max_epochs, hyperband_iterations=1, overwrite=True, directory=f'{ROOT_PATH}models/kt/v{kt_iter}/', project_name='_'.join(ticker_group))

tuner.search(train_inputs, train_labels, validation_data=(validation_inputs, validation_labels), epochs=epochs, verbose=1)

Trial 11 Complete [01h 22m 00s]
val_loss: 0.694129228591919

Best val_loss So Far: 0.6928533911705017
Total elapsed time: 10h 50m 56s

Search: Running Trial #12

Hyperparameter    |Value             |Best Value So Far 
superblock_fina...|2                 |4                 
lr                |6.7657e-05        |7.1545e-05        
comp2             |15                |26                
comp3             |17                |24                
comp5             |18                |10                
comp7             |4                 |10                
comp11            |4                 |5                 
comp13            |7                 |8                 
comp17            |5                 |5                 
comp19            |6                 |6                 
comp23            |4                 |5                 
comp29            |4                 |4                 
comp31            |4                 |4                 
comp37            |4                 |4 

### 324/30 v8 full_new_wsd

In [ ]:
model_no = '324'
kt_iter = '30'
ticker_group = ['wsd']
epochs = 20
max_epochs = 20
batch_size = 64

dataset_size = 'full_new_wsd'
shuffle_buffer_size = 2048
generator = False

train_inputs, train_labels = core.load_dataset_wsd_traintest(subset='training', dataset_size=dataset_size, ROOT_PATH='''J:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

validation_inputs, validation_labels = core.load_dataset_wsd_traintest(subset='validation', dataset_size=dataset_size, ROOT_PATH='''J:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

tuner = kt.Hyperband(hypermodel=core.model_324_kt, objective='val_loss', max_epochs=max_epochs, hyperband_iterations=1, overwrite=True, directory=f'{ROOT_PATH}models/kt/v{kt_iter}/', project_name='_'.join(ticker_group))

tuner.search(train_inputs, train_labels, validation_data=(validation_inputs, validation_labels), epochs=epochs, verbose=1)

Trial 29 Complete [01h 01m 07s]
val_loss: 0.6918690204620361

Best val_loss So Far: 0.691201388835907
Total elapsed time: 10h 30m 50s

Search: Running Trial #30

Hyperparameter    |Value             |Best Value So Far 
lr                |1.0189e-06        |2.4931e-05        
c_filters         |48                |48                
r_units           |64                |64                
tuner/epochs      |20                |20                
tuner/initial_e...|0                 |0                 
tuner/bracket     |0                 |0                 
tuner/round       |0                 |0                 

Epoch 1/20
5345/5345 [==============================] - 180s 34ms/step - loss: 0.6948 - accuracy: 0.5265 - val_loss: 0.6917 - val_accuracy: 0.5306
Epoch 4/20
5345/5345 [==============================] - 181s 34ms/step - loss: 0.6909 - accuracy: 0.5301 - val_loss: 0.6931 - val_accuracy: 0.5306
Epoch 6/20
5345/5345 [==============================] - 182s 34ms/step - loss: 0.6894 -

### Run with revised generator
- Use 326 as early benchmark model, with world portion variation to find the optimal point

In [ ]:
# RERUN WITH REVISED traintest slice code
model_no = '326'
constituent_limits = 0.025
id_constituent = 1
kt_iter = f'_model-{model_no}_wc-{constituent_limits}_ic-{id_constituent}'
ticker_group = ['wsd']
epochs = 20
max_epochs = 20
batch_size = 1024
shuffle_buffer_size = 16
generator = True

# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH='''J:\#PROJECT\idx''', db_ver='8', constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH='''J:\#PROJECT\idx''', db_ver='8', constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

tuner = kt.Hyperband(hypermodel=core.model_326_kt, objective='val_loss', max_epochs=max_epochs, hyperband_iterations=1, overwrite=True, directory=f'{ROOT_PATH}models/kt/v{kt_iter}/', project_name='_'.join(ticker_group))

tuner.search(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

Trial 5 Complete [00h 59m 40s]
val_loss: 0.6479974985122681

Best val_loss So Far: 0.6461509466171265
Total elapsed time: 04h 51m 48s

Search: Running Trial #6

Hyperparameter    |Value             |Best Value So Far 
lr                |4.8071e-07        |0.0003832         
r_units           |64                |96                
d_units_1         |64                |128               
d_units_2         |64                |96                
tuner/epochs      |3                 |3                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |2                 |2                 
tuner/round       |0                 |0                 

Epoch 1/3
1220/1220 [==============================] - 837s 674ms/step - loss: 0.9184 - accuracy: 0.5195 - val_loss: 0.9966 - val_accuracy: 0.5317
Epoch 2/3
1220/1220 [==============================] - 834s 674ms/step - loss: 0.8717 - accuracy: 0.5225 - val_loss: 0.9305 - val_accuracy: 0.5333
Epoch 3/3
 532/1220 [==========